In [3]:
#good one
import pandas as pd
from faker import Faker
import random
import string
from sqlalchemy import MetaData, Table, create_engine, Column, Integer, String, ForeignKey, Text, DateTime
fake = Faker()
num_rows = 1500000
def generate_username():
    username_length = 8
    characters = string.ascii_letters + string.digits
    return ''.join(random.choice(characters) for i in range(username_length)) + random.choice(string.digits) + random.choice(string.digits)
usernames = set(generate_username() for _ in range(num_rows))
def generate_random_id():
    characters = string.ascii_letters + string.digits
    id = ''.join(random.choice(characters) for _ in range(10))
    return id
def generate_wait_times():
    return pd.Timedelta(minutes=random.randint(1, 60))
def generate_category():
    return random.choice(["Bar", "Restaurant", "Hotel", "Stadium", "Park"])
def generate_product_category():
    return random.choice(["Outdoor", "Sports", "Clothing", "Kitchen", "Home Decor", "Bathroom"])
def generate_password():
    password_length = 15
    characters = string.ascii_letters + string.digits + string.punctuation
    return ''.join(random.choice(characters) for i in range(password_length))
def generate_fake_price():
    fake_price = fake.pydecimal(left_digits=3, right_digits=2, positive=True)
    return fake_price
# Create CSV files
csv_files = ['Friends.csv', 'Post.csv', 'UserProfile.csv', 'Product.csv', 'ProductReview.csv', 'Venue.csv', 'VenueReview.csv', 'UserTimeLine.csv' ]
for file_name in csv_files:
    if file_name == 'Friends.csv':
        data = {
            'FriendshipID': [generate_random_id() for _ in range(num_rows)],
            'Status': [random.randint(0, 2) for _ in range(num_rows)],
            'Username': list(usernames),
            'TargetedUsername': [generate_username() for _ in range(num_rows)],
            'StartTime': [fake.date() for _ in range(num_rows)],
        }
    elif file_name == 'Post.csv':
        data = {
            'PostID': [generate_random_id() for _ in range(num_rows)],
            'Username': list(usernames),
            'Comments': [fake.text() for _ in range(num_rows)],
            'Description': [fake.text() for _ in range(num_rows)],
            'PostCreatedAt': [fake.date() for _ in range(num_rows)],
           
            'PostLikes': [str(generate_wait_times()) for _ in range(num_rows)]
        }
    elif file_name == 'UserProfile.csv':
        data = {
            'Username': list(usernames),
            'FName': [fake.first_name() for _ in range(num_rows)],
            'LName': [fake.last_name() for _ in range(num_rows)],
            'Email': [fake.email() for _ in range(num_rows)],
            'Phone_number': [fake.phone_number() for _ in range(num_rows)],
            'Address': [fake.address() for _ in range(num_rows)],
            'DOB': [fake.date_of_birth(minimum_age=18, maximum_age=65).strftime('%Y-%m-%d') for _ in range(num_rows)],
            'PermLevel': [random.randint(0, 2) for _ in range(num_rows)],
            'Password': [generate_password() for _ in range(num_rows)],
            'Gender': [random.choice(['Male', 'Female']) for _ in range(num_rows)],
            'Biography': [fake.text(max_nb_chars=200) for _ in range(num_rows)]
        }
    elif file_name == 'Product.csv':
        data = {
            'ProductID': [generate_random_id() for _ in range(num_rows)],
            'ProductName': [fake.word() for _ in range(num_rows)],
            'ProdCategory': [generate_product_category() for _ in range(num_rows)],
            'PoductDescription': [fake.text() for _ in range(num_rows)],
            'Price': [generate_fake_price() for _ in range(num_rows)],
        }
    elif file_name == 'ProductReview.csv':
        data = {
            'ProductReviewID': [generate_random_id() for _ in range(num_rows)],
            'Username': list(usernames),
            'Comments': [fake.text() for _ in range(num_rows)],
            'PRDescription': [fake.text() for _ in range(num_rows)],
            'PRCreatedAt': [fake.date() for _ in range(num_rows)],
            'PicorVidID': [generate_random_id() for _ in range(num_rows)],
            'PRLikes': [str(generate_wait_times()) for _ in range(num_rows)]
        }
    elif file_name == 'Venue.csv':
        data = {
            'VenName': [fake.company() for _ in range(num_rows)],
            'Ven_Phone_number': [fake.phone_number() for _ in range(num_rows)],
            'Location': [fake.address() for _ in range(num_rows)],
            'Category': [generate_category() for _ in range(num_rows)],
        }
    elif file_name == 'VenueReview.csv':
        data = {
            'VenReviewID': [generate_random_id() for _ in range(num_rows)],
            'Username': list(usernames),
            'ReviewText': [fake.text() for _ in range(num_rows)],
            'PRDescription': [fake.text() for _ in range(num_rows)],
            'VRCreatedAt': [fake.date() for _ in range(num_rows)],
            'PicorVidID': [generate_random_id() for _ in range(num_rows)],
            'Likes': [str(generate_wait_times()) for _ in range(num_rows)],
            'Rating': [random.randint(1, 5) for _ in range(num_rows)],
            'Ven_Name': [fake.company() for _ in range(num_rows)],
            'WaitTimeReported': [generate_wait_times() for _ in range(num_rows)],
        }
    elif file_name == 'UserTimeLine.csv':
        data = {
            'UserTimeLineID': [generate_random_id() for _ in range(num_rows)],
            'PostID': [generate_random_id() for _ in range(num_rows)],
            'VenReviewID': [generate_random_id() for _ in range(num_rows)],
            'ProductReviewID': [generate_random_id() for _ in range(num_rows)],
            'Username': list(usernames),

        }

    df = pd.DataFrame(data)
    df.to_csv(file_name, index=False)

database_name = 'Bool'
engine = create_engine(f'sqlite:///{database_name}.db')
metadata = MetaData()

# Upload each CSV file to its corresponding table in the database
for csv_file in csv_files:
    table_name = csv_file.split('.')[0]
    df = pd.read_csv(csv_file)

    if table_name == 'UserProfile':
        table = Table(table_name, metadata,
                      Column('UserID', Integer, primary_key=True, autoincrement=True),
                      Column('FName', String),
                      Column('LName', String),
                      Column('Email', String),
                      Column('Phone_number', String),
                      Column('Address', String),
                      Column('DOB', String),
                      Column('PermLevel', Integer),
                      Column('Password', String),
                      Column('Gender', String),
                      Column('Username', String, unique=True),
                      Column('Biography', Text),
                      )
    elif table_name == 'Friends':
        table = Table(table_name, metadata,
                      Column('FriendshipID', Integer, primary_key=True, autoincrement=True),
                      Column('Status', Integer),
                      Column('Username', String, ForeignKey('UserProfile.Username')),
                      Column('TargetedUsername', String, ForeignKey('UserProfile.Username')),
                      Column('StartTime', String),
                      )
    elif table_name == 'Venue':
        table = Table(table_name, metadata,
                      Column('VenID', Integer, primary_key=True, autoincrement=True),
                      Column('Ven_Phone_number', String),
                      Column('Location', String),
                      Column('Category', String),
                      Column('VenName', String, unique=True),
                      )
    elif table_name == 'VenueReview':
        table = Table(table_name, metadata,
                      Column('VenReviewID', Integer, primary_key=True, autoincrement=True),
                      Column('Username', String, ForeignKey('UserProfile.Username')),
                      Column('ReviewText', Text),
                      Column('PRDescription', Text),
                      Column('VRCreatedAt', DateTime),
                      Column('Likes', Text),
                      Column('Rating', Integer),
                      )
    elif table_name == 'ProductReview':
        table = Table(table_name, metadata,
                      Column('ProductReviewID', Integer, primary_key=True, autoincrement=True),
                      Column('Username', String, ForeignKey('UserProfile.Username')),
                      Column('Comments', Text),
                      Column('PRDescription', Text),
                      Column('PRCreatedAt', DateTime),
                      Column('PRLikes', Text),
                      )
    elif table_name == 'Product':
        table = Table(table_name, metadata,
                      Column('ProductID', Integer, primary_key=True, autoincrement=True),
                      Column('ProductName', String, unique=True),
                      )
    elif table_name == 'Post':
        table = Table(table_name, metadata,
                      Column('PostID', Integer, primary_key=True, autoincrement=True),
                      Column('Username', String, ForeignKey('UserProfile.Username')),
                      Column('Comments', Text),
                      Column('Description', Text),
                      Column('PostCreatedAt', DateTime),
                      Column('WaitTimeReported', DateTime),
                      Column('PostLikes', Integer),
                      )
# Import CSV files into the database
for file_name in csv_files:
    table_name = file_name.split('.')[0]
    df = pd.read_csv(file_name)
    df.to_sql(table_name, engine, index=False, if_exists='replace')
print("Database and tables created successfully.")
#Prints first row of all tables
database_name = 'Bool'
engine = create_engine(f'sqlite:///{database_name}.db')
table_names = ['Friends', 'Post', 'UserProfile', 'Product', 'ProductReview', 'Venue', 'VenueReview']
for table_name in table_names:
    query = f'SELECT * FROM {table_name} LIMIT 1'
    result = pd.read_sql(query, engine)
    print(f'\nFirst line of table {table_name}:\n{result}')

Database and tables created successfully.

First line of table Friends:
  FriendshipID  Status    Username TargetedUsername   StartTime
0   dE1OGkNYHN       2  bZmo91Se30       Ut22BD0q12  2023-11-21

First line of table Post:
       PostID    Username                                           Comments  \
0  WnZsrVo5WC  bZmo91Se30  Head best toward less. Social carry day new ea...   

                                         Description PostCreatedAt  \
0  Full church provide government answer even. Te...    1978-03-25   

         PostLikes  
0  0 days 00:50:00  

First line of table UserProfile:
     Username      FName     LName                       Email   Phone_number  \
0  bZmo91Se30  Cassandra  Williams  petersonamanda@example.org  (473)331-3077   

                                             Address         DOB  PermLevel  \
0  480 Simpson Greens Apt. 128\nSouth Justintown,...  1958-07-02          0   

          Password  Gender                                          Biogr

In [ ]:
SELECT
    v.Category,
    AVG(CAST(vr.Likes AS REAL)) AS AvgLikes
FROM VenueReview vr
JOIN Venue v ON vr.Username = v.VenName
GROUP BY v.Category;

SELECT
    v.Category,
    AVG(vr.Rating) AS AvgRating
FROM VenueReview vr
JOIN Venue v ON vr.Username = v.VenName
GROUP BY v.Category;


SELECT
    v.Category,
    AVG(WaitTimeReported) AS AvgWaitTime
FROM Post p
JOIN ProductReview pr ON p.PostID = pr.PicorVidID
JOIN Venue v ON pr.Username = v.VenName
GROUP BY v.Category;


SELECT
    Username,
    COUNT(TargetedUsername) AS NumFriends
FROM Friends
WHERE Status = 2  -- Assuming '2' represents a confirmed friendship status
GROUP BY Username;


WITH UserFriends AS (
    SELECT DISTINCT
        f1.TargetedUsername AS FriendUsername
    FROM
        Friends f1
    WHERE
        f1.Status = 2  -- Assuming '2' represents a confirmed friendship status
)

SELECT DISTINCT
    fof.TargetedUsername AS PotentialFriend
FROM
    Friends f1
JOIN
    Friends f2 ON f1.TargetedUsername = f2.Username
JOIN
    UserFriends uf ON f2.TargetedUsername = uf.FriendUsername
JOIN
    Friends fof ON uf.FriendUsername = fof.Username
JOIN
    Post p1 ON f1.TargetedUsername = p1.Username
JOIN
    Post p2 ON fof.TargetedUsername = p2.Username AND p1.PostID = p2.PostID
WHERE
    f1.Username = 'your_username'  -- Replace 'your_username' with the target user
    AND f1.Status = 2  -- Assuming '2' represents a confirmed friendship status
    AND f2.Status = 2  -- Assuming '2' represents a confirmed friendship status
    AND fof.Status = 2  -- Assuming '2' represents a confirmed friendship status
    AND f1.TargetedUsername != 'your_username'  -- Exclude the user themselves
    AND fof.TargetedUsername NOT IN (
        SELECT
            TargetedUsername
        FROM
            Friends
        WHERE
            Username = 'your_username'  -- Exclude existing friends
    );

SELECT
    p.Description AS PostDescription,
    p.PostLikes AS NumberOfLikes,
    p.PostID,
    p.Username AS PosterUsername
FROM
    Post p
WHERE
    p.PostID = 'your_post_id';  -- Replace 'your_post_id' with the actual PostID you're interested in


SELECT
    v.VenName,
    strftime('%Y-%W', datetime(vr.VRCreatedAt)) AS Week,
    AVG(vr.Rating) AS AvgRatingPerWeek
FROM
    VenueReview vr
JOIN
    Venue v ON vr.Username = v.VenName
GROUP BY
    v.VenName, Week
ORDER BY
    v.VenName, Week;


SELECT
    v.Category,
    v.VenName,
    AVG(vr.Rating) AS AvgRating
FROM
    Venue v
LEFT JOIN
    VenueReview vr ON v.VenName = vr.Username
GROUP BY
    v.Category, v.VenName
ORDER BY
    v.Category, v.VenName;


SELECT
    p.PostID,
    p.Username,
    p.PostCreatedAt,
    strftime('%Y', 'now') - strftime('%Y', u.DOB) - (strftime('%m-%d', 'now') < strftime('%m-%d', u.DOB)) AS AgeAtPost
FROM
    Post p
JOIN
    UserProfile u ON p.Username = u.Username;

INSERT INTO UserProfile (FName, LName, Email, Phone_number, Address, DOB, PermLevel, Password, Gender, Username, Biography)
VALUES (
    'John',  -- Replace with a random first name
    'Doe',   -- Replace with a random last name
    'john.doe@example.com',  -- Replace with a random email
    '+1234567890',  -- Replace with a random phone number
    '123 Main St, Cityville',  -- Replace with a random address
    '1995-05-15',  -- Replace with a random date of birth
    1,  -- Replace with a random permission level
    'hashed_password',  -- Replace with a hashed password
    'Male',  -- Replace with a random gender
    'new_user123',  -- Replace with a random username
    'A new user added for testing purposes.'  -- Replace with a random biography
);

INSERT INTO Post (Username, Comments, Description, PostCreatedAt, WaitTimeReported, PostLikes)
VALUES (
    'new_user123',  -- Replace with a random username
    'A random comment.',  -- Replace with a random comment
    'A random post description.',  -- Replace with a random post description
    datetime('now'),  -- Use the current timestamp for the post creation time
    datetime('now', '+1 day'),  -- Replace with a random wait time reported
    datetime('now', '+2 days')  -- Replace with a random post likes timestamp
);

INSERT INTO VenueReview (Username, ReviewText, PRDescription, VRCreatedAt, Likes, Rating)
VALUES (
    'new_user123',  -- Replace with a random username
    'A random review text.',  -- Replace with a random review text
    'A random PR description.',  -- Replace with a random PR description
    datetime('now'),  -- Use the current timestamp for the review creation time
    datetime('now', '+3 days'),  -- Replace with a random likes timestamp
    4  -- Replace with a random rating
);
INSERT INTO ProductReview (Username, Comments, PRDescription, PRCreatedAt, ProductReviewID, PicorVidID, PRLikes)
VALUES (
    'new_user123',  -- Replace with a random username
    'A random comment.',  -- Replace with a random comment
    'A random PR description.',  -- Replace with a random PR description
    datetime('now'),  -- Use the current timestamp for the review creation time
    (SELECT randomblob(8)),  -- Replace with a random product review ID
    (SELECT randomblob(8)),  -- Replace with a random PicorVidID
    datetime('now', '+4 days')  -- Replace with a random PR likes timestamp
);

